In [1]:
# https://huggingface.co/microsoft/Phi-3-mini-128k-instruct/blob/main/sample_finetune.py

In [2]:
import sys
import logging

import datasets
from datasets import load_dataset
from peft import LoraConfig
import torch
import transformers
from trl import SFTTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from huggingface_hub import HfApi, HfFolder, Repository, create_repo

In [3]:
logger = logging.getLogger(__name__)

In [4]:
training_config = {
    "bf16": True,
    "do_eval": False,
    "learning_rate": 5.0e-06,
    "log_level": "info",
    "logging_steps": 20,
    "logging_strategy": "steps",
    "lr_scheduler_type": "cosine",
    "num_train_epochs": 1,
    "max_steps": -1,
    "output_dir": "./sample_phi3_finetune_example",
    "overwrite_output_dir": True,
    "per_device_eval_batch_size": 4,
    "per_device_train_batch_size": 4,
    "remove_unused_columns": True,
    "save_steps": 100,
    "save_total_limit": 1,
    "seed": 0,
    "gradient_checkpointing": True,
    "gradient_checkpointing_kwargs":{"use_reentrant": False},
    "gradient_accumulation_steps": 1,
    "warmup_ratio": 0.2,
    }

In [5]:
peft_config = {
    "r": 16,
    "lora_alpha": 32,
    "lora_dropout": 0.05,
    "bias": "none",
    "task_type": "CAUSAL_LM",
    "target_modules": "all-linear",
    "modules_to_save": None,
}

In [6]:
train_conf = TrainingArguments(**training_config)
peft_conf = LoraConfig(**peft_config)


In [7]:
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)
log_level = train_conf.get_process_log_level()
logger.setLevel(log_level)
datasets.utils.logging.set_verbosity(log_level)
transformers.utils.logging.set_verbosity(log_level)
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()

# Log on each process a small summary
logger.warning(
    f"Process rank: {train_conf.local_rank}, device: {train_conf.device}, n_gpu: {train_conf.n_gpu}"
    + f" distributed training: {bool(train_conf.local_rank != -1)}, 16-bits training: {train_conf.fp16}"
)
logger.info(f"Training/evaluation parameters {train_conf}")
logger.info(f"PEFT parameters {peft_conf}")

2024-06-03 13:27:31 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1 distributed training: True, 16-bits training: False
2024-06-03 13:27:31 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
batch_eval_metrics=False,
bf16=True,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulat

In [8]:
################
# Modle Loading
################
#checkpoint_path = "microsoft/Phi-3-mini-4k-instruct"
checkpoint_path = "microsoft/Phi-3-mini-128k-instruct"
model_kwargs = dict(
    use_cache=False,
    trust_remote_code=True,
    attn_implementation="flash_attention_2",  # loading the model with flash-attenstion support
    torch_dtype=torch.bfloat16,
    device_map=None
)
model = AutoModelForCausalLM.from_pretrained(checkpoint_path, **model_kwargs)
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
tokenizer.model_max_length = 512
tokenizer.pad_token = tokenizer.unk_token  # use unk rather than eos token to prevent endless generation
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
tokenizer.padding_side = 'right'

[INFO|configuration_utils.py:733] 2024-06-03 13:27:31,861 >> loading configuration file config.json from cache at C:\Users\acer alan\.cache\huggingface\hub\models--microsoft--Phi-3-mini-128k-instruct\snapshots\5be6479b4bc06a081e8f4c6ece294241ccd32dec\config.json
[INFO|configuration_utils.py:733] 2024-06-03 13:27:32,075 >> loading configuration file config.json from cache at C:\Users\acer alan\.cache\huggingface\hub\models--microsoft--Phi-3-mini-128k-instruct\snapshots\5be6479b4bc06a081e8f4c6ece294241ccd32dec\config.json
[INFO|configuration_utils.py:796] 2024-06-03 13:27:32,076 >> Model config Phi3Config {
  "_name_or_path": "microsoft/Phi-3-mini-128k-instruct",
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/Phi-3-mini-128k-instruct--configuration_phi3.Phi3Config",
    "AutoModelForCausalLM": "microsoft/Phi-3-mini-128k-instruct--modeling_phi3.Phi3ForCausalLM"
  },
  "bos_token_id": 1,
  

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[INFO|modeling_utils.py:4280] 2024-06-03 13:27:42,389 >> All model checkpoint weights were used when initializing Phi3ForCausalLM.

[INFO|modeling_utils.py:4288] 2024-06-03 13:27:42,390 >> All the weights of Phi3ForCausalLM were initialized from the model checkpoint at microsoft/Phi-3-mini-128k-instruct.
If your task is similar to the task the model of the checkpoint was trained on, you can already use Phi3ForCausalLM for predictions without further training.
[INFO|configuration_utils.py:917] 2024-06-03 13:27:42,640 >> loading configuration file generation_config.json from cache at C:\Users\acer alan\.cache\huggingface\hub\models--microsoft--Phi-3-mini-128k-instruct\snapshots\5be6479b4bc06a081e8f4c6ece294241ccd32dec\generation_config.json
[INFO|configuration_utils.py:962] 2024-06-03 13:27:42,642 >> Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": [
    32000,
    32001,
    32007
  ],
  "pad_token_id": 32000
}

[INFO|tokenization_utils_base.py:2108] 2024-06-03 

In [9]:
##################
# Data Processing
##################
def apply_chat_template(
    example,
    tokenizer,
):
    messages = example["messages"]
    # Add an empty system message if there is none
    if messages[0]["role"] != "system":
        messages.insert(0, {"role": "system", "content": ""})
    example["text"] = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=False)
    return example

raw_dataset = load_dataset("wenlianghuang/dataset_phi3_matt_testing")
train_dataset = raw_dataset["train_sft"]
test_dataset = raw_dataset["test_sft"]
column_names = list(train_dataset.features)

Overwrite dataset info from restored data version if exists.


2024-06-03 13:27:51 - INFO - datasets.builder - Overwrite dataset info from restored data version if exists.


Loading Dataset info from C:\Users\acer alan\.cache\huggingface\datasets/wenlianghuang___dataset_phi3_matt_testing/default/0.0.0/a1fb0788e4ae946b0027f6f1318161a2c26c4282


2024-06-03 13:27:51 - INFO - datasets.info - Loading Dataset info from C:\Users\acer alan\.cache\huggingface\datasets/wenlianghuang___dataset_phi3_matt_testing/default/0.0.0/a1fb0788e4ae946b0027f6f1318161a2c26c4282


Found cached dataset dataset_phi3_matt_testing (C:/Users/acer alan/.cache/huggingface/datasets/wenlianghuang___dataset_phi3_matt_testing/default/0.0.0/a1fb0788e4ae946b0027f6f1318161a2c26c4282)


2024-06-03 13:27:51 - INFO - datasets.builder - Found cached dataset dataset_phi3_matt_testing (C:/Users/acer alan/.cache/huggingface/datasets/wenlianghuang___dataset_phi3_matt_testing/default/0.0.0/a1fb0788e4ae946b0027f6f1318161a2c26c4282)


Loading Dataset info from C:/Users/acer alan/.cache/huggingface/datasets/wenlianghuang___dataset_phi3_matt_testing/default/0.0.0/a1fb0788e4ae946b0027f6f1318161a2c26c4282


2024-06-03 13:27:51 - INFO - datasets.info - Loading Dataset info from C:/Users/acer alan/.cache/huggingface/datasets/wenlianghuang___dataset_phi3_matt_testing/default/0.0.0/a1fb0788e4ae946b0027f6f1318161a2c26c4282


In [10]:
processed_train_dataset = train_dataset.map(
    apply_chat_template,
    fn_kwargs={"tokenizer": tokenizer},
    num_proc=5,
    remove_columns=column_names,
    desc="Applying chat template to train_sft",
)


Process #0 will write at C:\Users\acer alan\.cache\huggingface\datasets\wenlianghuang___dataset_phi3_matt_testing\default\0.0.0\a1fb0788e4ae946b0027f6f1318161a2c26c4282\cache-61068def4bd2125e_00000_of_00005.arrow


2024-06-03 13:27:51 - INFO - datasets.arrow_dataset - Process #0 will write at C:\Users\acer alan\.cache\huggingface\datasets\wenlianghuang___dataset_phi3_matt_testing\default\0.0.0\a1fb0788e4ae946b0027f6f1318161a2c26c4282\cache-61068def4bd2125e_00000_of_00005.arrow


Process #1 will write at C:\Users\acer alan\.cache\huggingface\datasets\wenlianghuang___dataset_phi3_matt_testing\default\0.0.0\a1fb0788e4ae946b0027f6f1318161a2c26c4282\cache-61068def4bd2125e_00001_of_00005.arrow


2024-06-03 13:27:51 - INFO - datasets.arrow_dataset - Process #1 will write at C:\Users\acer alan\.cache\huggingface\datasets\wenlianghuang___dataset_phi3_matt_testing\default\0.0.0\a1fb0788e4ae946b0027f6f1318161a2c26c4282\cache-61068def4bd2125e_00001_of_00005.arrow


Process #2 will write at C:\Users\acer alan\.cache\huggingface\datasets\wenlianghuang___dataset_phi3_matt_testing\default\0.0.0\a1fb0788e4ae946b0027f6f1318161a2c26c4282\cache-61068def4bd2125e_00002_of_00005.arrow


...

Spawning 5 processes


2024-06-03 13:27:51 - INFO - datasets.arrow_dataset - Spawning 5 processes


Applying chat template to train_sft (num_proc=5):   0%|          | 0/5000 [00:00<?, ? examples/s]

Concatenating 5 shards


2024-06-03 13:27:57 - INFO - datasets.arrow_dataset - Concatenating 5 shards


In [11]:
processed_test_dataset = test_dataset.map(
    apply_chat_template,
    fn_kwargs={"tokenizer": tokenizer},
    num_proc=5,
    remove_columns=column_names,
    desc="Applying chat template to test_sft",
)

Process #0 will write at C:\Users\acer alan\.cache\huggingface\datasets\wenlianghuang___dataset_phi3_matt_testing\default\0.0.0\a1fb0788e4ae946b0027f6f1318161a2c26c4282\cache-825ef8715f20ed9d_00000_of_00005.arrow


2024-06-03 13:27:57 - INFO - datasets.arrow_dataset - Process #0 will write at C:\Users\acer alan\.cache\huggingface\datasets\wenlianghuang___dataset_phi3_matt_testing\default\0.0.0\a1fb0788e4ae946b0027f6f1318161a2c26c4282\cache-825ef8715f20ed9d_00000_of_00005.arrow


Process #1 will write at C:\Users\acer alan\.cache\huggingface\datasets\wenlianghuang___dataset_phi3_matt_testing\default\0.0.0\a1fb0788e4ae946b0027f6f1318161a2c26c4282\cache-825ef8715f20ed9d_00001_of_00005.arrow


2024-06-03 13:27:57 - INFO - datasets.arrow_dataset - Process #1 will write at C:\Users\acer alan\.cache\huggingface\datasets\wenlianghuang___dataset_phi3_matt_testing\default\0.0.0\a1fb0788e4ae946b0027f6f1318161a2c26c4282\cache-825ef8715f20ed9d_00001_of_00005.arrow


Process #2 will write at C:\Users\acer alan\.cache\huggingface\datasets\wenlianghuang___dataset_phi3_matt_testing\default\0.0.0\a1fb0788e4ae946b0027f6f1318161a2c26c4282\cache-825ef8715f20ed9d_00002_of_00005.arrow


...

Spawning 5 processes


2024-06-03 13:27:57 - INFO - datasets.arrow_dataset - Spawning 5 processes


Applying chat template to test_sft (num_proc=5):   0%|          | 0/1000 [00:00<?, ? examples/s]

Concatenating 5 shards


2024-06-03 13:28:00 - INFO - datasets.arrow_dataset - Concatenating 5 shards


In [12]:
###########
# Training
###########
trainer = SFTTrainer(
    model=model,
    args=train_conf,
    peft_config=peft_conf,
    train_dataset=processed_train_dataset,
    eval_dataset=processed_test_dataset,
    max_seq_length=1024,
    dataset_text_field="text",
    tokenizer=tokenizer,
    packing=True
)
train_result = trainer.train()
metrics = train_result.metrics
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

Using custom data configuration default-8fe75ed2c3a85b51


2024-06-03 13:28:01 - INFO - datasets.builder - Using custom data configuration default-8fe75ed2c3a85b51


Loading Dataset Infos from c:\Users\acer alan\AppData\Local\Programs\Python\Python311\Lib\site-packages\datasets\packaged_modules\generator


2024-06-03 13:28:01 - INFO - datasets.info - Loading Dataset Infos from c:\Users\acer alan\AppData\Local\Programs\Python\Python311\Lib\site-packages\datasets\packaged_modules\generator


Generating dataset generator (C:/Users/acer alan/.cache/huggingface/datasets/generator/default-8fe75ed2c3a85b51/0.0.0)


...

{'loss': 1.0785, 'grad_norm': 0.2353515625, 'learning_rate': 4.056538994822945e-08, 'epoch': 0.95}


c:\Users\acer alan\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
[INFO|configuration_utils.py:733] 2024-06-03 14:52:06,506 >> loading configuration file config.json from cache at C:\Users\acer alan\.cache\huggingface\hub\models--microsoft--Phi-3-mini-128k-instruct\snapshots\5be6479b4bc06a081e8f4c6ece294241ccd32dec\config.json
[INFO|configuration_utils.py:796] 2024-06-03 14:52:06,509 >> Model config Phi3Config {
  "_name_or_path": "Phi-3-mini-128k-instruct",
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/Phi-3-mini-128k-instruct--configuration_phi3.Phi3Config",
    "AutoModelForCausalLM": "microsoft/Phi-3-mini-128k-instruct--modeling_ph

{'loss': 1.1849, 'grad_norm': 0.228515625, 'learning_rate': 2.2256479464999315e-08, 'epoch': 0.97}
{'loss': 1.1015, 'grad_norm': 0.32421875, 'learning_rate': 9.386068276959204e-09, 'epoch': 0.98}
{'loss': 1.1445, 'grad_norm': 0.17578125, 'learning_rate': 1.982406169283857e-09, 'epoch': 0.99}


[INFO|trainer.py:2329] 2024-06-03 14:56:01,058 >> 

Training completed. Do not forget to share your model on huggingface.co/models =)




{'train_runtime': 5262.9957, 'train_samples_per_second': 1.274, 'train_steps_per_second': 0.319, 'train_loss': 1.1665670079849415, 'epoch': 1.0}
***** train metrics *****
  epoch                    =         1.0
  total_flos               = 143808611GF
  train_loss               =      1.1666
  train_runtime            =  1:27:42.99
  train_samples_per_second =       1.274
  train_steps_per_second   =       0.319


In [13]:
#############
# Evaluation
#############
tokenizer.padding_side = 'left'
metrics = trainer.evaluate()
metrics["eval_samples"] = len(processed_test_dataset)
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

[INFO|trainer.py:3719] 2024-06-03 14:56:01,084 >> ***** Running Evaluation *****
[INFO|trainer.py:3721] 2024-06-03 14:56:01,085 >>   Num examples = 1328
[INFO|trainer.py:3724] 2024-06-03 14:56:01,085 >>   Batch size = 4


  0%|          | 0/332 [00:00<?, ?it/s]

***** eval metrics *****
  epoch                   =        1.0
  eval_loss               =     1.1452
  eval_runtime            = 0:05:02.28
  eval_samples            =       1000
  eval_samples_per_second =      4.393
  eval_steps_per_second   =      1.098


In [14]:
# ############
# # Save model
# ############
trainer.save_model(train_conf.output_dir)

[INFO|trainer.py:3410] 2024-06-03 15:01:03,428 >> Saving model checkpoint to ./sample_phi3_finetune_example
c:\Users\acer alan\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
[INFO|configuration_utils.py:733] 2024-06-03 15:01:04,276 >> loading configuration file config.json from cache at C:\Users\acer alan\.cache\huggingface\hub\models--microsoft--Phi-3-mini-128k-instruct\snapshots\5be6479b4bc06a081e8f4c6ece294241ccd32dec\config.json
[INFO|configuration_utils.py:796] 2024-06-03 15:01:04,276 >> Model config Phi3Config {
  "_name_or_path": "Phi-3-mini-128k-instruct",
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/Phi-3-mini-128k-instruct--

In [16]:
from huggingface_hub import notebook_login
notebook_login()

In [17]:
model.push_to_hub("sample_phi3_finetune_example",
                  use_auth_token=True,
                  commit_message="Training Phi-3",
                  private=True)

c:\Users\acer alan\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\hub.py:836: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
[INFO|configuration_utils.py:472] 2024-06-03 15:24:32,828 >> Configuration saved in sample_phi3_finetune_example\config.json
[INFO|configuration_utils.py:731] 2024-06-03 15:24:32,829 >> Configuration saved in sample_phi3_finetune_example\generation_config.json
[INFO|modeling_utils.py:2626] 2024-06-03 15:24:40,206 >> The model is bigger than the maximum size per checkpoint (5GB) and is going to be split in 2 checkpoint shards. You can find where each parameters has been saved in the index located at sample_phi3_finetune_example\model.safetensors.index.json.
[INFO|hub.py:759] 2024-06-03 15:24:45,366 >> Uploading the following files to wenlianghuang/sample_phi3_finetune_example: config.json,generation_config.json,model-00001-of-00002.safe

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.71G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/wenlianghuang/sample_phi3_finetune_example/commit/880aeee5a7b4893531bd3994d8aadab14056fa46', commit_message='Training Phi-3', commit_description='', oid='880aeee5a7b4893531bd3994d8aadab14056fa46', pr_url=None, pr_revision=None, pr_num=None)

In [22]:
device = torch.device('cuda' if torch.cuda.is_available()else 'cpu')
print(device)

cuda


In [23]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# Define the prompt and prepare the input
prompt = "Write a step-by-step recipe on how to make a vegan black bean soup, including all necessary ingredients and cooking instructions. Please also include any variations or substitutions for dietary restrictions."
tokenizer = AutoTokenizer.from_pretrained("wenlianghuang/sample_phi3_finetune_example")
inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)

# Check if CUDA (GPU) is available and move model and inputs to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the model weights from hub and move to the appropriate device
model_id = "wenlianghuang/sample_phi3_finetune_example"
model = AutoModelForCausalLM.from_pretrained(model_id)
trained_model = PeftModel.from_pretrained(model, model_id)
trained_model.to(device)

# Move inputs to the same device as the model
inputs = {key: value.to(device) for key, value in inputs.items()}

# Run inference
outputs = trained_model.generate(**inputs, max_length=1000)

# Decode the outputs
text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
print(text)


[INFO|tokenization_auto.py:666] 2024-06-03 16:38:01,896 >> Could not locate the tokenizer configuration file, will try to use the model config instead.
c:\Users\acer alan\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/3.60k [00:00<?, ?B/s]

[INFO|configuration_utils.py:733] 2024-06-03 16:38:02,317 >> loading configuration file config.json from cache at C:\Users\acer alan\.cache\huggingface\hub\models--wenlianghuang--sample_phi3_finetune_example\snapshots\e5c922a3b7d6c050381aca3ec1c728746e13689b\config.json
[INFO|configuration_utils.py:796] 2024-06-03 16:38:02,319 >> Model config Phi3Config {
  "_name_or_path": "wenlianghuang/sample_phi3_finetune_example",
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/Phi-3-mini-128k-instruct--configuration_phi3.Phi3Config",
    "AutoModelForCausalLM": "microsoft/Phi-3-mini-128k-instruct--modeling_phi3.Phi3ForCausalLM"
  },
  "bos_token_id": 1,
  "embd_pdrop": 0.0,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "model_type": "phi3",
  "num_attention_heads": 32,
  "num_h

OSError: Can't load tokenizer for 'wenlianghuang/sample_phi3_finetune_example'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'wenlianghuang/sample_phi3_finetune_example' is the correct path to a directory containing all relevant files for a LlamaTokenizerFast tokenizer.